# Step 4: Test and Compare Models

In this step, you'll test your trained model and compare it with EasyOCR's pre-trained model to evaluate performance.

## What You'll Learn:
- How to set up a custom OCR model for inference
- How to compare custom vs pre-trained models
- How to interpret OCR results and confidence scores

## 4.1 Setup: Copy Trained Model

First, copy your trained model to the `user_network_dir` directory where EasyOCR will load it.

**Important Files Needed:**
- `user_network_dir/custom.pth` - Your trained model weights
- `user_network_dir/custom.yaml` - Model configuration (architecture, character list)
- `user_network_dir/custom.py` - Model architecture definition

In [1]:
from pathlib import Path
import shutil

# Create the user network directory if it doesn't exist
user_network_dir = Path("./user_network_dir")
user_network_dir.mkdir(parents=True, exist_ok=True)

# Copy your trained model to the user network directory
# ⚠️ IMPORTANT: Change the folder name below to match your training run
source_model = Path("./saved_models/TPS-VGG-BiLSTM-CTC-Seed1111-20251119-024440/best_accuracy.pth")
target_model = user_network_dir / "custom.pth"

if not source_model.exists():
    raise FileNotFoundError(f"Expected trained model at {source_model} but it was not found.")

shutil.copy2(source_model, target_model)
print(f"Copied {source_model} -> {target_model}")

Copied saved_models\TPS-VGG-BiLSTM-CTC-Seed1111-20251119-024440\best_accuracy.pth -> user_network_dir\custom.pth


## 4.2 Prepare Test Images

Place your test images in the `demo_images` folder. These should be images similar to what your model will process in production (e.g., receipts, invoices, signs).

```bash
demo_images/
  ├── demo_image_01.jpg
  ├── demo_image_02.jpg
  └── ...
```

## 4.3 Test with Pre-trained Model (Baseline)

First, test with EasyOCR's pre-trained model to establish a baseline performance.

In [1]:
%pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.5 MB/s eta 0:00:00
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 52.2 MB/s eta 0:00:00:00:0100:01
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 62.9 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 64.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with pre-trained traditional Chinese model
print("Loading EasyOCR pre-trained model...")
reader_pretrained = Reader(['ch_tra'], gpu=True)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_pretrained.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading EasyOCR pre-trained model...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete
TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)

filename: 'demo_image_01.jpg', confidence: 0.5216, string: '九龍青衣細山路2號美景花園平臺72號舖'


## 4.4 Test with Your Custom Model

Now test with your trained model. Make sure you have:
- ✅ `user_network_dir/custom.pth` (model weights)
- ✅ `user_network_dir/custom.yaml` (configuration)
- ✅ `user_network_dir/custom.py` (architecture)

In [3]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with your custom model
print("Loading your custom trained model...")
reader_custom = Reader(
    ['ch_tra'], 
    gpu=True,
    model_storage_directory='./user_network_dir',
    user_network_directory='./user_network_dir',
    recog_network='custom'
)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH YOUR CUSTOM MODEL")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_custom.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading your custom trained model...

TESTING WITH YOUR CUSTOM MODEL

filename: 'demo_image_01.jpg', confidence: 0.0356, string: 't  l $謳'


## 4.5 Side-by-Side Comparison

Compare both models on the same images to see which performs better.

In [4]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize both models
print("Loading both models...")
reader_pretrained = Reader(['ch_tra'], gpu=True)
reader_custom = Reader(['ch_tra'], gpu=True,
                      model_storage_directory='./user_network_dir',
                      user_network_directory='./user_network_dir',
                      recog_network='custom')

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*80}")
print("SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model")
print(f"{'='*80}\n")

# Compare results for each image
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    print(f"📷 Testing: {filename}")
    print(f"{'-'*80}")
    
    # Test with pre-trained model
    results_pretrained = reader_pretrained.readtext(file_path)
    print("🔵 EasyOCR Pre-trained Model:")
    if results_pretrained:
        for bbox, text, confidence in results_pretrained:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print()
    
    # Test with custom model
    results_custom = reader_custom.readtext(file_path)
    print("🟢 Your Custom Model:")
    if results_custom:
        for bbox, text, confidence in results_custom:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print(f"\n{'='*80}\n")

Loading both models...

SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model

📷 Testing: demo_image_01.jpg
--------------------------------------------------------------------------------
🔵 EasyOCR Pre-trained Model:
   confidence: 0.0148, string: '九龍青衣紐山路2號美景花園平壼72號舖'

🟢 Your Custom Model:
   confidence: 0.0356, string: 't  l $謳'


